![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Prompt Notebook with Chat - Prompt Lab Notebook v1.1.0
This notebook contains steps and code to demonstrate inferencing of prompts
generated in Prompt Lab in watsonx.ai with a chat format. It introduces Python API commands
for authentication using API key and prompt inferencing using WML API.

**Note:** Notebook code generated using Prompt Lab will execute successfully.
If code is modified or reordered, there is no guarantee it will successfully execute.
For details, see: <a href="/docs/content/wsj/analyze-data/fm-prompt-save.html?context=wx" target="_blank">Saving your work in Prompt Lab as a notebook.</a>

Some familiarity with Python is helpful. This notebook uses Python 3.10.

## Notebook goals
The learning goals of this notebook are:

* Defining a Python function for obtaining credentials from the IBM Cloud personal API key
* Defining parameters of the Model object
* Using the Model object to generate response using the defined model id, parameters and the prompt input

# Setup

In [1]:
!pip install --upgrade 'chromadb==0.3.26' 'pydantic==1.10.0' sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.4/138.4 kB 6.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 4.9 MB/s eta 0:00:00


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 60.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 977.4/977.4 kB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 31.3 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.4/436.4 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 73.7 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 68.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 65.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 76.2 MB/s eta 0:00:00:00:

Successfully built hnswlib
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.4.0
    Uninstalling typing_extensions-4.4.0:
      Successfully uninstalled typing_extensions-4.4.0
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2022.11.0
    Uninstalling fsspec-2022.11.0:
      Successfully uninstalled fsspec-2022.11.0


## watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud personal API key. For details, see
<a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank">documentation</a>.


In [12]:
import os
import getpass

def get_credentials():
	return {
		"url" : "https://eu-de.ml.cloud.ibm.com",
		"apikey" : getpass.getpass("JM59wATYpa21z8IWGcTPwlwFFR39xZwGSNNASVPuCCdA")
	}


# Inferencing
This cell demonstrated how we can use the model object as well as the created access token
to pair it with parameters and input string to obtain
the response from the the selected foundation model.

## Defining the model id
We need to specify model id that will be used for inferencing:


In [13]:
model_id = "sdaia/allam-1-13b-instruct"


## Defining the model parameters
We need to provide a set of model parameters that will influence the
result:

In [14]:
parameters = {
    "decoding_method": "sample",
    "max_new_tokens": 700,
    "temperature": 0.52,
    "top_k": 46,
    "top_p": 1,
    "repetition_penalty": 1
}

## Defining the project id or space id
The API requires project id or space id that provides the context for the call. We will obtain
the id from the project or space in which this notebook runs:

In [15]:
project_id = os.getenv("PROJECT_ID")
space_id = os.getenv("SPACE_ID")


## Defining the Model object
We need to define the Model object using the properties we defined so far:


In [16]:
from ibm_watsonx_ai.foundation_models import Model

model = Model(
	model_id = model_id,
	params = parameters,
	credentials = get_credentials(),
	project_id = project_id,
	space_id = space_id
	)


JM59wATYpa21z8IWGcTPwlwFFR39xZwGSNNASVPuCCdA········


## Defining the vector index
Initialize the vector index to query when chatting with the model.

In [17]:
from ibm_watsonx_ai.client import APIClient

wml_credentials = get_credentials()
client = APIClient(credentials=wml_credentials, project_id=project_id, space_id=space_id)

vector_index_id = "46a44af2-e5e9-42df-bfb1-66b8bd4aeba0"
vector_index_details = client.data_assets.get_details(vector_index_id)
vector_index_properties = vector_index_details["entity"]["vector_index"]

from ibm_watsonx_ai.foundation_models.embeddings.sentence_transformer_embeddings import SentenceTransformerEmbeddings

emb = SentenceTransformerEmbeddings('sentence-transformers/all-MiniLM-L6-v2')

import subprocess
import gzip
import json
import chromadb
import random
import string

def hydrate_chromadb():
    data = client.data_assets.get_content(vector_index_id)
    content = gzip.decompress(data)
    stringified_vectors = str(content, "utf-8")
    vectors = json.loads(stringified_vectors)

    chroma_client = chromadb.Client()

    # make sure collection is empty if it already existed
    collection_name = "my_collection"
    try:
        collection = chroma_client.delete_collection(name=collection_name)
    except:
        print("Collection didn't exist - nothing to do.")
    collection = chroma_client.create_collection(name=collection_name)

    vector_embeddings = []
    vector_documents = []
    vector_metadatas = []
    vector_ids = []

    for vector in vectors:
        vector_embeddings.append(vector["embedding"])
        vector_documents.append(vector["content"])
        metadata = vector["metadata"]
        lines = metadata["loc"]["lines"]
        clean_metadata = {}
        clean_metadata["asset_id"] = metadata["asset_id"]
        clean_metadata["asset_name"] = metadata["asset_name"]
        clean_metadata["url"] = metadata["url"]
        clean_metadata["from"] = lines["from"]
        clean_metadata["to"] = lines["to"]
        vector_metadatas.append(clean_metadata)
        asset_id = vector["metadata"]["asset_id"]
        random_string = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))
        id = "{}:{}-{}-{}".format(asset_id, lines["from"], lines["to"], random_string)
        vector_ids.append(id)

    collection.add(
        embeddings=vector_embeddings,
        documents=vector_documents,
        metadatas=vector_metadatas,
        ids=vector_ids
    )
    return collection

chroma_collection = hydrate_chromadb()

def proximity_search( question ):
    query_vectors = emb.embed_query(question)
    query_result = chroma_collection.query(
        query_embeddings=query_vectors,
        n_results=vector_index_properties["settings"]["top_k"],
        include=["documents", "metadatas", "distances"]
    )

    documents = list(reversed(query_result["documents"][0]))

    return "\n".join(documents)


JM59wATYpa21z8IWGcTPwlwFFR39xZwGSNNASVPuCCdA········


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Collection didn't exist - nothing to do.


## Defining the inferencing input for chat
Foundation models supporting chat accept a system prompt that instructs the model on how to conduct the dialog. They also accept previous questions and answers to give additional context when inferencing. Each model has it's own string format for constructing the input.

Let us provide the input we got from the Prompt Lab and format it for the selected model:


In [18]:
prompt_input = """
__grounding__

<<SYS>>
أنت مدير لعبة تفاعلية لتعلم حروف الجر العربية مع اللاعب عن طريق المحادثة. اللعبة تركز على إملاء فراغات الجمل بحرف الجر المناسب، التأكد من صحة اختيارات اللاعب، حساب نقاطه، متابعة قائمة حروف الجر العشوائية والحذف منها.
اللعبة تتألف من 3 مستويات:
المستوى المبتدئ : أحرف الجر: مِنْ، إلَى، عَنْ، عَلَى، فِي. طول القائمة: 9 أحرف جر. النقاط: 3 نقاط يحسم منها نقطة عند كل خطأ.
المستوى المتوسط : أحرف الجر: مِنْ، إلَى، عَنْ، عَلَى، فِي،ب، ل، ك. طول القائمة: 25 حرف جر. النقاط: 10 نقاط يحسم منها نقطة عند كل خطأ.
المستوى المتقدم: أحرف الجر: مِنْ، إلَى، عَنْ، عَلَى، فِي،ب، ل، ك، ت، مُذْ، مُنْذُ، رُبَّ، حَتَّى. طول القائمة: 25 حرف جر. النقاط: 5 نقاط يحسم منها نقطة عند كل خطأ.
قائمة حروف الجر العشوائية: عليك توليد قائمة من حروف الجر  يجب أن تحتوي كل الحروف التي تدخل في المستوى المحدد وكرر الأحرف عشوائياً حتى تصبح القائمة بطول 25 حرف جر.
التقييم: قم بتقييم إجابة اللاعب. إذا كانت صحيحة، احذف حرف الجر الصحيح من قائمة حروف الجر العشوائية وقدم للاعب جملة جديدة. أما إذا كانت إجابته خاطئة، احسم من رصيد نقاطه نقطة واحد، فيصبح مجموع نقاطه هو المجموع السابق ناقص نقطة واحدة..
 الفوز والخسارة: سيفوز اللاعب باللعبة عندما يتمكن من شطب جميع أحرف الجر من القائمة، أو عندما يخبرك أنه "شطب" عموداً أو سطراً أو قطراً. ويخسر اللعبة إذا وصل رصيد نقاطه إلى الصفر.
اللعبة:
تبدأ اللعبة بأن يخبرك اللاعب باسمه.
أجب اللاعب بأن ترحب به ب5 كلمات فقط، واطلب منه تحديد أي الثلاث مستويات يريد أن يلعب.
يكتب اللاعب "مبتدئ" أو "متوسط" أو "متقدم"، بعدها أخبره عن قواعد الحروف التي تدخل في المستوى الذي اختاره وشرح لكل حرف باستخدام 9 كلمات. وأخبره أن يكتب "أبدأ" لتبدأ اللعبة.
عندما يكتب اللاعب "أبدأ" قم بتوليد وطباعة قائمة أحرف الجر العشوائية التي تدخل في المستوى المختار بالطول المحدد للقائمة حسب المستوى المحدد.
عندما يكتب اللاعب "جملة"، قم بطباعة جملة باللغة العربية صحيحة النحو بها فراغ يملئ بحرف جر، ثم اطبع له ما تبقى من قائمة أحرف الجر العشوائية التي أعددتها في البداية.
أطلب من اللاعب أن يختار حرف الجر المناسب والصحيح من قائمة أحرف الجر العشوائية ليملأ الفراغ.
عندما يختار اللاعب حرف الجر المناسب من قائمة أحرف الجر العشوائية ليملأ الفراغ، تأكد من إجابته بحيث يكون المعنى صحيحاً والجملة سليمة من الناحية النحوية والقواعدية.
أخبر اللاعب إذا كانت إجابته خاطئة واحسم له نقطة من النقاط، أي عدد النقاط الحالي ناقص واحد، وأوقف اللعبة في حال وصل عدد النقاط إلى صفر 0.
أخبر اللاعب إذا كانت إجابته صحيحة، واحذف وجوداًَ واحداً لكل حرف صحيح يختار اللاعب من قائمة أحرف الجر العشوائية.
أعد طباعة قائمة أحرف الجر العشوائية وانتظر كلمة "جملة" ليكتبها اللاعب لتقوم بطباعة سؤال جديد.
 يستمر اللعب حتى يكتب اللاعب "شطب" بمعنى أنه شطب عمود أو سطر أو قطر فيفوز، أو حتى يخسر بأن يكون عدد نقاطه صفر 0.
<</SYS>>

 سنا [/INST]  أهلاً وسهلاً بك في اللعبة التفاعلية لتعلم حروف الجر العربية. سأرشدك خلال الخطوات باللعبة. يُرجى اتباع التعليمات التالية:

1. ابدأ بإخباري باسمك باستخدام الصيغة التالية: اسمي [اسمك]. مثلاً: اسمي أحمد.

2. حدد المستوى الذي ترغب في اللعب به بكتابة 'مبتدئ'، 'متوسط'، أو 'متقدم'.

3. بعد تحديد المستوى، سأشرح لك قواعد حروف الجر وبعض الأمثلة لمساعدتك على فهم استخدامها.

4. عندما تكون جاهزًا لبدء اللعبة، اكتب 'أبدأ' للمتابعة.

أتطلع إلى الاستمتاع بتعلم حروف الجر العربية معاً!  </s><s> [INST] سنا، مبتدئ [/INST]  أهلاً وسهلاً بك يا سنا في المستوى المبتدئ للعبة! في هذا المستوى، سنتعلم أربعة حروف جر أساسية هي: مِن، إلى، عن، وعلى. سأساعدك على فهم استخداماتها وبعض الأمثلة.

1. مِن (مِنْ): يُستخدم للدلالة على الزمان والمكان والسببية.
   مثال للزمان: صُمتُ مِن الفجر (أي من وقت الفجر).
   مثال للمكان: جئتُ مِن البيت (أي من مكان البيت).
   مثال للسببية: اكتسبتُ هذه المعلومة مِن الكتاب (أي بسبب الكتاب).

2. إلى (إلى): يُستخدم للدلالة على الانتهاء المكاني والزماني.
   مثال للانتهاء المكاني: وصلتُ إلى المطار (أي وصلت لنقطة المطار).
   مثال للانتهاء الزماني: سأزوركَ إلى الغد (أي حتى الغد).

3. عن (عَنْ): يُستخدم للدلالة على البُعد أو الانعطاف.
   مثال: تحدثتُ عن الموضوع (أي حول الموضوع).

4. على (على): يُستخدم للدلالة على الاستعلاء والظروف.
   مثال للاستعلاء: الكتابُ على الطاولة (أي موضوع فوق الطاولة).
   مثال للظروف: مشيتُ على الرصيف (أي بجانب الرصيف).

عندما تكون جاهزًا لبدء اللعبة، اكتب "أبدأ" للمتابعة.  </s><s> [INST] أبدأ [/INST]  مرحبًا بك يا سنا في بداية اللعبة! سأقوم الآن بإنشاء قائمة عشوائية من حروف الجر المناسبة للمستوى المبتدئ. تتألف هذه القائمة من أربعة حروف جر التي تعلمناها للتو: مِن، إلى، عن، وعلى.

فيما يلي قائمة حروف الجر العشوائية للمستوى المبتدئ:

1. مِن
2. إلى
3. عن
4. على

عندما تكون جاهزًا للعب، اكتب "جملة" لأقدم لك سؤالاً مع فراغ يجب ملؤه بحرف جر مناسب.  </s><s> [INST] جملة [/INST]  أهلا بك يا سنا! إليكم سؤالاً مع فراغ يجب ملؤه بحرف الجر المناسب من القائمة العشوائية التي قمنا بإنشائها للمستوى المبتدئ:

الجملة: ذهبتُ ___(1)___ المدرسةِ.

يرجى اختيار حرف الجر المناسب من القائمة التالية:
1. مِن
2. إلى
3. عن
4. على

أرجو منك كتابة رقم حرف الجر المناسب بين الأقواس (1، 2، 3، 4) لإكمال الجملة.  </s><s> [INST]"""


## Execution
Let us now use the defined Model object, pair it with the input, and generate the response to your question:


In [20]:
user_input = input("اسمك: ")
while user_input != "توقف":
    grounding = proximity_search(user_input)
    formattedQuestion = f"""<s> [INST] {user_input} [/INST]"""
    prompt = f"""{prompt_input}{formattedQuestion}"""
    generated_response = model.generate_text(prompt=prompt.replace("__grounding__", grounding), guardrails=False)
    print(f"AI: {generated_response}")
    user_input = input("إجابتك: ")


اسمك: سنا
AI:  أهلا بك يا سنا مجدداً! يبدو أنك لم تختر حرف الجر المناسب لإكمال الجملة. دعنا نُكمل اللعبة معًا:

الجملة: ذهبتُ ___(1)___ المدرسةِ.

يرجى اختيار حرف الجر المناسب من القائمة التالية:
1. مِن
2. إلى
3. عن
4. على

أرجو منك كتابة رقم حرف الجر المناسب بين الأقواس (1، 2، 3، 4) لإكمال الجملة. 
إجابتك: لنبدأ لعبة جديدة، المستوى المتوسط
AI:  مرحبا بك يا سنا! سعيد لسماع رغبتك في الانتقال إلى المستوى المتوسط. في هذا المستوى، ستتعلم خمسة حروف جر إضافية بجانب الأربعة التي تعلمتها في المستوى المبتدئ. هذه الحروف هي: ب، ل، ك، في، وت.

قبل البدء، دعنا نستعرض استخدام هذه الحروف:

1. ب (ب): يُستخدم للدلالة على الأداة أو السبب.
مثال: رسمتُ اللوحة بـ (بواسطة) الفرشاة.

2. ل (ل): يُستخدم للدلالة على الملكية أو الغرض.
مثال: الكتاب لـ (لـ) الطالب.

3. ك (ك): يُستخدم للدلالة على المقارنة أو المشابهة.
مثال: هذا الوجه يشبه القمر كـ (كما).

4. في (في): يُستخدم للدلالة على المكان أو الظرف.
مثال: نمتُ في السرير (أي داخل السرير).

5. ت (ت): حرف جر نادر الاستخدام ولا يعتبر حرف جر بالمعنى الدقيق، لكنه يست

# Next steps
You successfully completed this notebook! You learned how to use
watsonx.ai inferencing SDK to generate response from the foundation model
based on the provided input, model id and model parameters. Check out the
official watsonx.ai site for more samples, tutorials, documentation, how-tos, and blog posts.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2023 IBM. This notebook and its source code are released under the terms of the ILAN License.
Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for watsonx.ai Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for Watson Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="https://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF" target="_blank">License Terms</a>  